# Refer to 
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/01.train-within-notebook.ipynb
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/11.production-deploy-to-aks.ipynb

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [2]:
import azureml.core
print(azureml.core.VERSION)

0.1.4


In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [4]:
#Creating the model pickle file
import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet152(weights='imagenet')

Using TensorFlow backend.


In [5]:
model

In [6]:
model_json = model.to_json()
with open("model_resnet.json", "w") as json_file:
    json_file.write(model_json)

In [7]:
model.save_weights("model_resnet_weights.h5")

In [8]:
#from keras.models import model_from_json
#model = model_from_json(model_json)

In [9]:
model.load_weights('model_resnet_weights.h5')

In [10]:
model

In [11]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "model_resnet_weights.h5", # this points to a local file
                       model_name = "resnet_model", # this is the name the model is registered as
                       tags = ["dl", "resnet"],
                       description = "resnet 152 model",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model resnet_model
resnet_model resnet 152 model 10


In [12]:
#define init() function
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')

In [13]:
init()

Model loaded


In [14]:
#define run() function 
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))

In [15]:
import wget
wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

'220px-Lynx_lynx_poing (4).jpg'

In [16]:
img_path = '220px-Lynx_lynx_poing.jpg'

In [17]:
run(img_path)

Model loaded
Predicted: [[('n02127052', 'lynx', 0.9959085), ('n02128385', 'leopard', 0.0011504436), ('n02123159', 'tiger_cat', 0.0009417995)]]


In [18]:
%%writefile score.py
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')
    
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))    

Overwriting score.py


In [19]:
%run score.py

In [38]:
%%writefile resnet152_n.py
# -*- coding: utf-8 -*-
"""ResNet152 model for Keras.

# Reference:

- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

Adaptation of code from flyyufelix, mvoelk, BigMoyan, fchollet at https://github.com/adamcasson/resnet152

"""

import numpy as np
import warnings

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import add
from keras.models import Model
import keras.backend as K
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras import initializers
from keras.engine import Layer, InputSpec
from keras.preprocessing import image
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape

import sys
sys.setrecursionlimit(3000)

WEIGHTS_PATH = 'https://github.com/adamcasson/resnet152/releases/download/v0.1/resnet152_weights_tf.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/adamcasson/resnet152/releases/download/v0.1/resnet152_weights_tf_notop.h5'

class Scale(Layer):
    """Custom Layer for ResNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:

        out = in * gamma + beta,

    where 'gamma' and 'beta' are the weights and biases larned.

    Keyword arguments:
    axis -- integer, axis along which to normalize in mode 0. For instance,
        if your input tensor has shape (samples, channels, rows, cols),
        set axis to 1 to normalize per feature map (channels axis).
    momentum -- momentum in the computation of the exponential average 
        of the mean and standard deviation of the data, for 
        feature-wise normalization.
    weights -- Initialization weights.
        List of 2 Numpy arrays, with shapes:
        `[(input_shape,), (input_shape,)]`
    beta_init -- name of initialization function for shift parameter 
        (see [initializers](../initializers.md)), or alternatively,
        Theano/TensorFlow function to use for weights initialization.
        This parameter is only relevant if you don't pass a `weights` argument.
    gamma_init -- name of initialization function for scale parameter (see
        [initializers](../initializers.md)), or alternatively,
        Theano/TensorFlow function to use for weights initialization.
        This parameter is only relevant if you don't pass a `weights` argument.
        
    """
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma'%self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta'%self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity_block is the block that has no conv layer at shortcut
    
    Keyword arguments
    input_tensor -- input tensor
    kernel_size -- defualt 3, the kernel size of middle conv layer at main path
    filters -- list of integers, the nb_filters of 3 conv layer at main path
    stage -- integer, current stage label, used for generating layer names
    block -- 'a','b'..., current block label, used for generating layer names
    
    """
    eps = 1.1e-5
    
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1
    
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size), name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = add([x, input_tensor], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut
    
    Keyword arguments:
    input_tensor -- input tensor
    kernel_size -- defualt 3, the kernel size of middle conv layer at main path
    filters -- list of integers, the nb_filters of 3 conv layer at main path
    stage -- integer, current stage label, used for generating layer names
    block -- 'a','b'..., current block label, used for generating layer names
        
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    
    """
    eps = 1.1e-5
    
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1
    
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), strides=strides, name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
                      name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, (1, 1), strides=strides,
                             name=conv_name_base + '1', use_bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = add([x, shortcut], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def ResNet152(include_top=True, weights=None,
              input_tensor=None, input_shape=None,
              large_input=False, pooling=None,
              classes=1000):
    """Instantiate the ResNet152 architecture.
    
    Keyword arguments:
    include_top -- whether to include the fully-connected layer at the 
        top of the network. (default True)
    weights -- one of `None` (random initialization) or "imagenet" 
        (pre-training on ImageNet). (default None)
    input_tensor -- optional Keras tensor (i.e. output of `layers.Input()`)
        to use as image input for the model.(default None)
    input_shape -- optional shape tuple, only to be specified if 
        `include_top` is False (otherwise the input shape has to be 
        `(224, 224, 3)` (with `channels_last` data format) or 
        `(3, 224, 224)` (with `channels_first` data format). It should 
        have exactly 3 inputs channels, and width and height should be 
        no smaller than 197. E.g. `(200, 200, 3)` would be one valid value.
        (default None)
    large_input -- if True, then the input shape expected will be 
        `(448, 448, 3)` (with `channels_last` data format) or 
        `(3, 448, 448)` (with `channels_first` data format). (default False)
    pooling -- Optional pooling mode for feature extraction when 
        `include_top` is `False`.
        - `None` means that the output of the model will be the 4D 
            tensor output of the last convolutional layer.
        - `avg` means that global average pooling will be applied to 
            the output of the last convolutional layer, and thus
            the output of the model will be a 2D tensor.
        - `max` means that global max pooling will be applied.
        (default None)
    classes -- optional number of classes to classify image into, only 
        to be specified if `include_top` is True, and if no `weights` 
        argument is specified. (default 1000)
            
    Returns:
    A Keras model instance.
        
    Raises:
    ValueError: in case of invalid argument for `weights`,
        or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    
    eps = 1.1e-5
    
    if large_input:
        img_size = 448
    else:
        img_size = 224
    
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=img_size,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)
    
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # handle dimension ordering for different backends
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1
            
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    if large_input:
        x = AveragePooling2D((14, 14), name='avg_pool')(x)
    else:
        x = AveragePooling2D((7, 7), name='avg_pool')(x)
    
    # include classification layer by default, not included for feature extraction 
    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
    
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet152')
    
    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('resnet152_weights_tf.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='cdb18a2158b88e392c0905d47dcef965')
        else:
            weights_path = get_file('resnet152_weights_tf_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='4a90dcdafacbd17d772af1fb44fc2660')
        model.load_weights(weights_path, by_name=True)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1000')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')
                
        if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
            warnings.warn('You are using the TensorFlow backend, yet you '
                          'are using the Theano '
                          'image data format convention '
                          '(`image_data_format="channels_first"`). '
                          'For best performance, set '
                          '`image_data_format="channels_last"` in '
                          'your Keras config '
                          'at ~/.keras/keras.json.')
    return model

if __name__ == '__main__':
    model = ResNet152(include_top=True, weights='imagenet')
    
    img_path = '220px-Lynx_lynx_poing.jpg'
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))

Writing resnet152_n.py


In [39]:
%run resnet152_n.py

Input image shape: (1, 224, 224, 3)
Predicted: [[('n02127052', 'lynx', 0.9959085), ('n02128385', 'leopard', 0.0011504436), ('n02123159', 'tiger_cat', 0.0009417995), ('n02124075', 'Egyptian_cat', 0.00051911507), ('n02128757', 'snow_leopard', 0.0003321905)]]


In [23]:
!more score.py

def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predi
ctions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')
    
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
m--More--(46%)

In [24]:
!more conda_dependencies.yml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-fil
e-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - --index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E75
01C02541B433786111FE8E140CAA1
  - --extra-index-url https://pypi.python.org/simple
  - azureml-defaults
  - papermill==0.14.1
  - python-dotenv==0.9.0
  - Pillow==5.2.0
  - wget==3.2
m--More--(86%)

In [25]:
!more resnet152.py

# -*- coding: utf-8 -*-
"""ResNet152 model for Keras.

# Reference:

- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.0338
5)

Adaptation of code from flyyufelix, mvoelk, BigMoyan, fchollet at https://github
.com/adamcasson/resnet152

"""

import numpy as np
import warnings

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
m--More--(3%)

In [28]:
import os
os.getcwd()

'/home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow'

In [40]:
os.listdir(os.getcwd())

['00_DevelopModel.ipynb',
 'AML_02_DevelopModelDriver_AKS-old.ipynb',
 'AML_01_trainlocal-docker.ipynb',
 'model_resnet_weights.h5',
 'AML_02_resnet_model_weights.ipynb',
 '01_DevelopModelDriver.ipynb',
 '__pycache__',
 'resnet152_n.py',
 '06_SpeedTestWebApp.ipynb',
 '03_TestLocally.ipynb',
 'AML_02_Develop_init_run_functions.ipynb',
 'AML_02_AKS_deployment.ipynb',
 'aml_config',
 '05_TestWebApp.ipynb',
 '04_DeployOnAKS.ipynb',
 'score.py',
 'AML_01_trainlocal.ipynb',
 'model_resnet.json',
 '02_BuildImage.ipynb',
 '07_TearDown.ipynb',
 'helpers.py',
 'testing_utilities.py',
 '220px-Lynx_lynx_poing.jpg',
 'conda_dependencies.yml',
 'resnet152.py',
 'README.md',
 'environment.yml',
 'AML_00_configuration.ipynb',
 'sample_projects',
 '.ipynb_checkpoints']

In [42]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "conda_dependencies.yml",
                                                  description = "Image for AKS Deployment Tutorial",
                                                  tags = ["AKS","AML"],
                                                  dependencies = ["resnet152_n.py"]
                                                 )

image = ContainerImage.create(name = "myimage1",
                              # this is the model object
                              models = [],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running.................................................
SucceededImage creation operation finished for image myimage1:4, operation "Succeeded"


In [43]:
#list images
images = ws.images()
images

[ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=myimage1, id=myimage1:4, tag=['AKS', 'AML'], version=4),
 ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=myimage1, id=myimage1:3, tag=['AKS', 'AML'], version=3),
 ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=myimage1, id=myimage1:2, tag=['AKS', 'AML'], version=2),
 ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=myimage1, id=myimage1:1, tag=['AKS', 'AML'], version=1),
 ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=my-aci-svc-1, id=my-aci-svc-1:4, tag=None, version=4),
 ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=my-aci-svc-1, id=my-aci-svc-1:3, tag=None, version=3),
 ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fda68124b70>, name=my-aci-svc-1, id=m

In [33]:
#images.delete()

In [46]:
#Provision AKS cluster
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'jaya-aks-1' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

ComputeTargetException: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-failure-cause': 'gateway', 'x-ms-request-id': '042fe68d-c97e-4343-9ed2-78d88d32ecf5', 'x-ms-correlation-request-id': '042fe68d-c97e-4343-9ed2-78d88d32ecf5', 'x-ms-routing-request-id': 'EASTUS:20180918T174521Z:042fe68d-c97e-4343-9ed2-78d88d32ecf5', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Content-Type-Options': 'nosniff', 'Date': 'Tue, 18 Sep 2018 17:45:21 GMT', 'Content-Length': '246'}
Content: b'{"error":{"code":"LinkedInvalidPropertyId","message":"Property id \'\' at path \'properties.resourceId\' is invalid. Expect fully qualified resource Id that start with \'/subscriptions/{subscriptionId}\' or \'/providers/{resourceProviderNamespace}/\'."}}'

In [36]:
#!az aks delete --resource-group jayavienna --name jaya-aks-1 --yes

In [45]:
%%time
aks_target.wait_for_provisioning(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

NameError: name 'aks_target' is not defined

In [ ]:
#Deploy web service to AKS
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [ ]:
%%time
aks_service_name ='jaya-aks-service-2'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
#Debug

# load workspace from default config file
ws = Workspace.from_config()

# list all web services in the workspace
for s in ws.webservices():
    print(s.name)

# instantiate the web service object from workspace and service name
svc = Webservice(workspace = ws, name = 'jaya-aks-service-2')

# print out the Docker log
print(svc.get_logs())

In [34]:
#clean up resources
aks_target = AksCompute(name='jaya-aks-1',workspace=ws)
aks_target.delete()

ComputeTargetException: ComputeTargetNotFound: Compute Target with name jaya-aks-1 not found in provided workspace

In [ ]:
for s in ws.webservices():
    print(s.name)

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget

ws = Workspace.from_config()

for c in ws.compute_targets():
    print(c.name)
